# Bridging gaps: The value of academic research for data scientists

## How to implement it: Application to the 2022 Olympic Winter Games
Now that we've seen the theoretical basis behind the ResNet, it's time for an application! While many machine learning models are implemented in well-known Python libraries, the ResNet for time series classification is not. No need to panic! We will not have to optimise our loss functions by hand, we can just rely on a deep learning framework, such as TensorFlow or PyTorch. To get a feel for the power of the ResNet, we will now try to predict whether a country has won a medal at the Olympic Winter Games in 2022 just by looking at their GDP from 1990 to 2018.

Let's start by importing the necessary libraries. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import tensorflow as tf
import tensorflow.keras as keras 

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, classification_report

import joblib
import random

### Preparing the Data
The data we use was collected from the web (https://olympics.com/en/olympic-games/beijing-2022/medals and https://www.kaggle.com/datasets/nitishabharathi/gdp-per-capita-all-countries) and put into files that can easily be read by Python. Let's start by opening these files. 

In [2]:
data = pd.read_csv("datasets/GDP.csv")
data.rename(columns={"Country ": "Country"}, inplace=True)
data["Country"] = data["Country"].str.strip()

In [3]:
medals = open("datasets/medals.txt").read().splitlines()[1:]
participation = open("datasets/participation.txt").read().splitlines()[1:]
countries = [i.strip() for i in open("datasets/countries.txt").read().splitlines()[1:]]

Not every country participated in the Olympic Winter Games of 2022. Let's therefore make a variable that indicates whether that country participated and one that indicates whether they won a medal. We only include countries that participated. In addition, the ResNet requires full time series without missing values. For the purpose of this example, we choose to delete 18 cases with missing values. 

In [320]:
data["medal"] = np.where(data["Country"].isin(medals), 1, 0)
data["participated"] = np.where(data["Country"].isin(participation), 1, 0)
data.drop(columns='2019',inplace=True)
data.dropna(inplace=True)
data.reset_index(inplace=True)
data = data[data["participated"]==1]

data.head()

,index,Country,Country Code,1990,1991,1992,1993,1994,1995,1996,...,2011,2012,2013,2014,2015,2016,2017,2018,medal,participated
1,3,Albania,ALB,2549.473022,1909.114038,1823.307673,2057.449657,2289.873135,2665.764906,2980.066288,...,10207.752350,10526.235450,10571.010650,11259.225890,11662.030480,11868.178970,12930.140030,13364.15540,0,1
4,6,Argentina,ARG,7380.115031,8210.643432,8942.569853,9777.214005,10435.910770,10225.118710,10857.429670,...,19817.450480,19764.225010,20365.613350,20008.320640,20551.833190,20130.408030,20843.155070,20610.56855,0,1
5,7,Armenia,ARM,2428.558960,2237.752728,1356.210786,1296.178498,1429.102386,1591.894846,1742.734114,...,7019.767748,7649.061531,8003.087763,8405.073655,8727.385447,8808.572714,9620.818491,10343.17559,0,1
7,9,Australia,AUS,17329.706610,17790.980140,18189.378740,19131.841870,20064.459010,20894.397210,21972.052650,...,41965.358420,42826.789580,45902.047950,46880.220660,46276.150690,47305.880020,49628.811810,51663.36509,1,1
8,10,Austria,AUT,19442.312040,20585.019310,21259.636750,21698.307670,22606.829640,23660.408610,24529.207830,...,44452.732750,46457.345780,47922.049120,48799.715470,49879.266470,51809.513630,53937.066380,55454.68929,1,1


Our data, consisting of 71 countries, now needs to be split into test and training data. Since we want to predict medal winners, the variable 'medal' becomes our target. GDP from 1990 to 2018 represents the features used for this prediction. Since most countries do not win a medal, there is some class imbalance. We therefore stratify our splits to make sure both our training and test sample contain the same percentage of medal winning countries. 

In [191]:
y = data["medal"].to_numpy().reshape(-1, 1)

x = data.loc[:, "1990":"2018"].to_numpy()
x = x.reshape((x.shape[0], x.shape[1], 1))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, shuffle=True, random_state=0)

### Training the Model
We are now ready to build and train the model. Before we set out the structure of the ResNet, we first set some parameters, such as the number of epochs and the batch size. 

In [192]:
n_feature_maps = 64 #number of filters
input_shape = x_train.shape[1:] 
nb_classes = 1 #there is only one final node 
nb_epochs = 1500 #1500 epochs
mini_batch_size = 10 #a batch size of 10
output_directory = os.getcwd()

In [193]:
input_layer = keras.layers.Input(input_shape)

#BLOCK 1
#layer 1
conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

#layer 2
conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

#layer 3
conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

#add raw input to output of layer 3
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

output_block_1 = keras.layers.add([shortcut_y, conv_z])
output_block_1 = keras.layers.Activation('relu')(output_block_1)

#BLOCK 2
#layer 1
conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

#layer 2
conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

#layer 3
conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

#add output of block 1 to output of layer 3
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

output_block_2 = keras.layers.add([shortcut_y, conv_z])
output_block_2 = keras.layers.Activation('relu')(output_block_2)

#BLOCK 3
#layer 1
conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

#layer 2
conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

#layer 3
conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

#add output of block 2 to output of layer 3
shortcut_y = keras.layers.BatchNormalization()(output_block_2)

output_block_3 = keras.layers.add([shortcut_y, conv_z])
output_block_3 = keras.layers.Activation('relu')(output_block_3)

#FINAL
#global average pooling
gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

#final dense layer
output_layer = keras.layers.Dense(nb_classes, activation='sigmoid')(gap_layer)

#specify and compile model
model = keras.models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.SGD(learning_rate=0.0001),
              metrics=['binary_accuracy', 'AUC'])

Now our model is compiled, we are ready to train it. Before we do so, we introduce some callbacks. More specifically, we save the best model in terms of training AUC at the end of each epoch. In addition, we allow for early stopping if the training loss does not decline any further. Afterwards, we train our model on the data. Because of the substantial randomness involved in setting the original weights, we set seeds to allow reproduction of our results. 

In [194]:
file_path = output_directory + '\\best_model.h5py'
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='auc',
                                                    save_best_only=True, mode='max')

early_stopping = keras.callbacks.EarlyStopping(monitor="loss", min_delta=0.00001, patience=50, mode="min")

callbacks = [model_checkpoint, early_stopping]

In [ ]:
np.random.seed(98566)
tf.random.set_seed(451246)
hist = model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=nb_epochs,
                verbose=0, validation_data=(x_test, y_test), callbacks=callbacks)

model.save(output_directory + '\\last_model.h5py')

### Interpreting the results
However good the model may be, without some sense-making it remains a black box. Let's first start by inspecting its performance. Throughout the epochs, the model should become better at matching the data. The figure shows that both the training and test loss decline througout the epochs. After about 350 epochs, our training stops since training loss has not decreased for several epochs. We also plot the test loss, yet do not make decisions based on this quantity. As expected, the test loss is always slightly higher than the training loss. Both training and test accuracy rapidly rise after the first 50 epochs and then remain almost constant at 80-85%. The AUC follows a similar pattern and remains constant at around 0.9. Overall, the model performs well. Since the accuracy of a classification problem is highly dependent on the chosen threshold, we choose our best model based on the training AUC. This is maximal around 210 epochs. 

In [198]:
hist_df = pd.DataFrame(hist.history)
joblib.dump(hist_df, "history.pkl")

In [ ]:
range = np.arange(1, len(hist_df)+1)

plt.style.use('bmh')
fig, ax = plt.subplots(3, 1, sharex="col", figsize=(5,13))
ax[0].plot(range, hist_df["loss"], label='Test Loss', color="steelblue")
ax[0].plot(range, hist_df["val_loss"], label='Training Loss', color="darkslateblue")
ax[0].vlines(hist_df.loss.idxmin(axis=0), 0, 1, color="crimson")
ax[0].text(hist_df.loss.idxmin(axis=0) - 10, 0.05, f"Min Train Loss ({round(min(hist_df.loss), 2)})", rotation="vertical")
ax[0].vlines(hist_df.val_loss.idxmin(axis=0), 0, 1, color="crimson")
ax[0].text(hist_df.val_loss.idxmin(axis=0) - 10, 0.05, f"Min Test Loss ({round(min(hist_df.val_loss), 2)})", rotation="vertical")
ax[0].set_ylim(0,1)
ax[0].set_ylabel("Loss")

ax[1].plot(range, hist_df["binary_accuracy"], label='Test Accuracy', color="steelblue")
ax[1].plot(range, hist_df["val_binary_accuracy"], label='Training Accuracy', color="darkslateblue")
ax[1].vlines(hist_df.binary_accuracy.idxmax(axis=0), 0, 1, color="crimson")
ax[1].text(hist_df.binary_accuracy.idxmax(axis=0) - 10, 0.05, f"Max Train Accuracy ({round(max(hist_df.binary_accuracy), 2)})", rotation="vertical")
ax[1].vlines(hist_df.val_binary_accuracy.idxmax(axis=0), 0, 1, color="crimson")
ax[1].text(hist_df.val_binary_accuracy.idxmax(axis=0) - 10, 0.05, f"Max Test Accuracy ({round(max(hist_df.val_binary_accuracy), 2)})", rotation="vertical")
ax[1].set_ylim(0,1)
ax[1].set_ylabel("Accuracy")

ax[2].plot(range, hist_df["auc"], label='Test', color="steelblue")
ax[2].plot(range, hist_df["val_auc"], label='Training', color="darkslateblue")
ax[2].vlines(hist_df.auc.idxmax(axis=0), 0, 1, color="crimson")
ax[2].text(hist_df.auc.idxmax(axis=0) - 10, 0.05, f"Max Train AUC ({round(max(hist_df.auc), 2)})", rotation="vertical")
ax[2].vlines(hist_df.val_auc.idxmax(axis=0), 0, 1, color="crimson")
ax[2].text(hist_df.val_auc.idxmax(axis=0) - 10, 0.05, f"Max Test AUC ({round(max(hist_df.val_auc), 2)})", rotation="vertical")
ax[2].set_ylim(0,1)
ax[2].legend()
ax[2].set_ylabel("AUC")
ax[2].set_xlabel("Epoch")

fig.savefig("results.png")

<p align = "center">
<img src="figures/results.png"/>
</p>
<p align = "center">
Performance of the model in terms of loss, accuracy and AUC for both the test and training set
</p>

In [200]:
best_model = keras.models.load_model("best_model.h5py")
y_pred = model.predict(x_train)

For the best model, we subsequently plot the ROC curve. With an AUC of 0.91 it performs significantly better than a random model. We choose the best threshold as the point on the curve that is closest to the upper left corner in terms of Euclidean distance, where the true positive rate is 1 and the false positive rate is 0. This point is indicated with a red dot. At this point, if we classify a country as a medal winner only if its predicted probability is above 0.54, we are able to achieve a false positive rate of 0.09 and a true positive rate of 0.76.

In [201]:
fpr, tpr, thresholds = roc_curve(y_train, y_pred)
auc = auc(fpr, tpr)

In [209]:
distances = [np.sqrt(i**2 + (1-j)**2) for i, j in zip(fpr, tpr)]
idx = distances.index(min(distances))

In [ ]:
plt.style.use('bmh')
fig, ax = plt.subplots(figsize=(5,5))
ax.plot([0, 1], [0, 1], 'k--', color="crimson")
ax.plot(fpr, tpr, color="steelblue")
ax.scatter(fpr[idx], tpr[idx], s=500, color="crimson", marker="o")
ax.text(fpr[idx] - 0.1, tpr[idx] + 0.1, f"({round(fpr[idx], 2)}, {round(tpr[idx], 2)})")
ax.text(0.6, 0.2, f"AUC = {round(auc, 2)}")
ax.set_xlabel('False positive rate')
ax.set_ylabel('True positive rate')
ax.set_ylim(0,1.05)
ax.set_xlim(-0.05,1)

fig.savefig("roc.png")

<p align = "center">
<img src="figures/roc.png"/>
</p>
<p align = "center">
ROC curve for the model with the best training AUC
</p>

In [221]:
thresholds[idx]

0.5440258

To present an unbiased estimate of our model's performance, we calculate various statistics on the test set. Our model achieves an overall precision of about 85% and a recall of about 87%.

In [225]:
y_test_pred = np.where(model.predict(x_test) > thresholds[idx], 1, 0)
print(classification_report(y_test, y_test_pred, target_names=["No medal", "Medal"]))

              precision    recall  f1-score   support

    No medal       0.92      0.86      0.89        14
       Medal       0.78      0.88      0.82         8

    accuracy                           0.86        22
   macro avg       0.85      0.87      0.86        22
weighted avg       0.87      0.86      0.87        22



#### Class Activation Map

Our model performs well in terms in terms of almost all metrics presented above. Yet, as with many deep learning models, it remains a black box. To find out what features of the time series add to its prediction, we can use a Class Activation Map. Before its final dense layer, the ResNet contains a Global Average Pooling (GAP) layer. Therefore, for a given time series with length $T$, let $S_k(t)$ represent the output (activation) of filter $k$ in the last convolutional layer for time point $t$. For this filter $k$, the output of the GAP layer is $f_k = \sum_{t=1}^T S_k(t)$. If we let $w_k$ represent the weight of this output in the final dense layer, the input for the final softmax function is 

$g = \sum_{k=1}^K w_k \sum_{t=1}^T S_k(t) = \sum_{k=1}^K \sum_{t=1}^T w_k S_k(t)$

The importance of each temporal element $t$ for the classification to class 1 is therefore  $\sum_{k=1}^K w_k S_k(t)$. For the purposes of the Class Activation Map, we can ignore the softmax activation in this case, since it is a monotone transformation. 

In [236]:
input = best_model.input
output_before_GAP = best_model.layers[35].output
new_model = keras.models.Model(input, output_before_GAP)
tmp_pred = new_model.predict(x_test) #predict the S_k(t)

weights = best_model.layers[37].get_weights()[0].reshape((1, 1, 128)) #get the weights
cam = np.multiply(tmp_pred, weights) #multiply each S_k(t) by corresponding w_k
cam = np.sum(cam, axis=2) #sum over all k


In [ ]:
x_test = x_test.reshape((22, 29))

fig, ax = plt.subplots(figsize=(7, 5))

inxval = np.array(data.loc[:, "1990":"2018"].columns.astype('int32'))

for i in np.arange(0, x_test.shape[0]): 
    value = x_test[i, :]
    color = cam[i, :]

    points = np.array([inxval, value]).T.reshape(-1,1,2)
    segments = np.concatenate([points[:-1],points[1:]], axis=1)

    lc = LineCollection(segments, cmap="coolwarm", linewidth=3)
    # set color to date values
    lc.set_array(color)
    ax.add_collection(lc)

ax.set_xlabel('Year')
ax.set_ylabel('GDP')

ax.autoscale_view()

fig.savefig("cam.png")

<p align = "center">
<img src="figures/cam.png"/>
</p>
<p align = "center">
Class Activation Map for the test set (high values are indicated by a deep red, low values by a deep blue)
</p>

Our class activation map shows the importance of each time point for the classification of that time series. It shows that especially the periods of high growth contribute to a higher probability of winning a medal at the 2022 Winter Olympics. Intuitively this makes sense: Wealthier countries have more funds to invest in the development of athletic capabilities and are therefore more likely to win a medal. 